In [ ]:
import pygame
import random
import sys

# Initialize
pygame.init()
WIDTH, HEIGHT = 400, 600
SCREEN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Flappy Bird")

# Load assets
BG = pygame.image.load("K:/assets/background-day.png")
BIRD_IMG = pygame.image.load("K:/assets/flappy.ico")
PIPE_IMG = pygame.image.load("K:/assets/pipe-red.png")
BASE_IMG = pygame.image.load("K:/assets/base.png")
WING_SOUND = pygame.mixer.Sound("K:/assets/sfx_wing.mp3")
POINT_SOUND = pygame.mixer.Sound("K:/assets/sfx_point.mp3")
HIT_SOUND = pygame.mixer.Sound("K:/assets/flappy-bird-hit-sound-101soundboards.mp3")

FONT = pygame.font.SysFont("Arial", 40)

# Game Constants
GRAVITY = 0.4
FLAP_POWER = -8
PIPE_GAP = 150
PIPE_SPEED = 3
BASE_Y = HEIGHT - 100

clock = pygame.time.Clock()

class Bird(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = BIRD_IMG
        self.rect = self.image.get_rect(center=(100, HEIGHT//2))
        self.movement = 0

    def update(self):
        self.movement += GRAVITY
        self.rect.y += self.movement

    def flap(self):
        self.movement = FLAP_POWER
        WING_SOUND.play()

class Pipe(pygame.sprite.Sprite):
    def __init__(self, x, y, flipped):
        super().__init__()
        self.image = pygame.transform.flip(PIPE_IMG, False, flipped)
        self.rect = self.image.get_rect(midtop=(x, y) if not flipped else (x, y - PIPE_IMG.get_height()))

    def update(self):
        self.rect.x -= PIPE_SPEED
        if self.rect.right < 0:
            self.kill()

def draw_base(x):
    SCREEN.blit(BASE_IMG, (x, BASE_Y))
    SCREEN.blit(BASE_IMG, (x + BASE_IMG.get_width(), BASE_Y))

def display_score(score):
    score_text = FONT.render(str(score), True, (255,255,255))
    SCREEN.blit(score_text, (WIDTH//2 - score_text.get_width()//2, 50))

def main():
    bird_group = pygame.sprite.GroupSingle(Bird())
    pipe_group = pygame.sprite.Group()

    base_x = 0
    score = 0
    can_score = True
    SPAWNPIPE = pygame.USEREVENT
    pygame.time.set_timer(SPAWNPIPE, 1200)

    running = True
    while running:
        SCREEN.blit(BG, (0, 0))
        base_x = (base_x - PIPE_SPEED) % BASE_IMG.get_width()
        draw_base(base_x)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                bird_group.sprite.flap()
            if event.type == SPAWNPIPE:
                height = random.randint(150, 400)
                bottom = Pipe(WIDTH, height, False)
                top = Pipe(WIDTH, height - PIPE_GAP, True)
                pipe_group.add(bottom, top)

        bird_group.update()
        pipe_group.update()

        bird_group.draw(SCREEN)
        pipe_group.draw(SCREEN)

        # Collision
        if pygame.sprite.spritecollide(bird_group.sprite, pipe_group, False) or bird_group.sprite.rect.top <= 0 or bird_group.sprite.rect.bottom >= BASE_Y:
            HIT_SOUND.play()
            break

        # Score
        for pipe in pipe_group:
            if pipe.rect.centerx < bird_group.sprite.rect.centerx and can_score:
                score += 0.5  # Each pipe pair counts once
                if score % 1 == 0:
                    POINT_SOUND.play()
                can_score = False
            if pipe.rect.centerx > bird_group.sprite.rect.centerx:
                can_score = True

        display_score(int(score))
        pygame.display.update()
        clock.tick(60)

    game_over(score)

def game_over(score):
    text = FONT.render(f"Game Over! Score: {int(score)}", True, (255, 255, 255))
    SCREEN.blit(text, (WIDTH//2 - text.get_width()//2, HEIGHT//2))
    pygame.display.update()
    pygame.time.wait(3000)
    main()

if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
